## [--- Day 9: Movie Theater ---](https://adventofcode.com/2025/day/9)

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import ClassVar, Iterator


@dataclass(frozen=True, slots=True, order=True)
class vector2:
    """Crude 2D vector wrapper for tuples"""

    x: int = 0
    y: int = 0

    zero: ClassVar["vector2"]
    up: ClassVar["vector2"]
    down: ClassVar["vector2"]
    left: ClassVar["vector2"]
    right: ClassVar["vector2"]

    def rotated_left(self) -> "vector2":
        return vector2(self.y, -self.x)

    def rotated_right(self) -> "vector2":
        return vector2(-self.y, self.x)

    def reflected_around(self, origin: "vector2") -> "vector2":
        return vector2(-self.x + 2 * origin.x, -self.y + 2 * origin.y)

    def distance_to(self, other: "vector2") -> int:
        return abs(self.x - other.x) + abs(self.y - other.y)

    def rect_area(self, other: "vector2") -> int:
        return (abs(self.x - other.x) + 1) * (abs(self.y - other.y) + 1)

    def __iter__(self) -> Iterator[int]:
        yield self.x
        yield self.y

    def __add__(self, other: "vector2") -> "vector2":
        return vector2(self.x + other.x, self.y + other.y)

    def __sub__(self, other: "vector2") -> "vector2":
        return vector2(self.x - other.x, self.y - other.y)

    def __repr__(self) -> str:
        return f"({self.x}, {self.y})"


vector2.zero = vector2(0, 0)
vector2.up = vector2(0, -1)
vector2.down = vector2(0, 1)
vector2.left = vector2(-1, 0)
vector2.right = vector2(1, 0)


class Grid:
    def __init__(self, data: list[vector2]) -> None:
        self.data = data

        xs: list[int] = [p.x for p in self.data]
        ys: list[int] = [p.y for p in self.data]
        self.min_x, self.max_x = min(xs), max(xs)
        self.min_y, self.max_y = min(ys), max(ys)

        self.width: int = self.max_x - self.min_x + 1
        self.height: int = self.max_y - self.min_y + 1

    def in_bounds(self, pos: vector2) -> bool:
        return self.min_x <= pos.x <= self.max_x and self.min_y <= pos.y <= self.max_y

    def rect_area(self, a: vector2, b: vector2) -> int:
        return (abs(a.x - b.x) + 1) * (abs(a.y - b.y) + 1)

In [ ]:
def solve() -> None:
    with open("..\\data\\09.txt") as file:
        coords = [line.strip().split(",") for line in file.readlines()]

    grid = Grid([vector2(int(x), int(y)) for x, y in coords])

    # Horizons
    north_east: list[vector2] = []
    south_east: list[vector2] = []
    south_west: list[vector2] = []
    north_west: list[vector2] = []

    # Lexicographical ascending sort, x then y
    grid.data.sort()
    max_y: int = 0
    for pos in grid.data:
        if pos.y > max_y:
            north_east.append(pos)
            max_y = pos.y

    min_y: int = grid.height
    for pos in reversed(grid.data):
        if pos.y < min_y:
            south_west.append(pos)
            min_y = pos.y

    # Lexicographical sort, x descending then y ascending
    grid.data.sort(key=lambda pos: (-pos.x, pos.y))
    max_y = 0
    for pos in grid.data:
        if pos.y > max_y:
            north_west.append(pos)
            max_y = pos.y

    min_y = grid.height
    for pos in reversed(grid.data):
        if pos.y < min_y:
            south_east.append(pos)
            min_y = pos.y

    non_dominated_points = set(north_east + south_east + south_west + north_west)
    dominated_points = set(grid.data) - non_dominated_points

    print(f"{len(dominated_points)} dominated points removed")
    print(f"{len(non_dominated_points)} non-dominated points remain\n")

    ne_sw_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    ne_sw_area: int = 0

    for ne in north_east:
        for sw in south_west:
            area = ne.rect_area(sw)
            if area > ne_sw_area:
                ne_sw_area = area
                ne_sw_pair = (ne, sw)

    print(f"NE-SW Max Area: {ne_sw_area} between {ne_sw_pair[0]} and {ne_sw_pair[1]}")

    se_nw_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    se_nw_area: int = 0

    for se in south_east:
        for nw in north_west:
            area = se.rect_area(nw)
            if area > se_nw_area:
                se_nw_area = area
                se_nw_pair = (se, nw)

    print(f"SE-NW Max Area: {se_nw_area} between {se_nw_pair[0]} and {se_nw_pair[1]}")
    print(f"The largest area between any two points is {max(ne_sw_area, se_nw_area)}")


solve()

280 dominated points removed
216 non-dominated points remain

NE-SW Max Area: 4678664144 between (16361, 84880) and (83532, 15229)
SE-NW Max Area: 4761736832 between (17804, 13266) and (86507, 82573)
The largest area between any two points is 4761736832


In [ ]:
class Compressed_Grid:
    def __init__(self, points: list[vector2]) -> None:
        self.points = points
        self.xs = sorted(list(set(p.x for p in points)))
        self.ys = sorted(list(set(p.y for p in points)))
        self.x_map: dict[int, int] = {x: i for i, x in enumerate(self.xs)}
        self.y_map: dict[int, int] = {y: i for i, y in enumerate(self.ys)}
        width = len(self.xs) - 1
        height = len(self.ys) - 1

        # Find inside/outside using scanline algorithm
        cells: list[list[bool]] = [[False] * width for _ in range(height)]
        for i in range(len(self.points)):
            p1 = self.points[i]
            p2 = self.points[(i + 1) % len(self.points)]

            # Draw line from p1 to p2
            if p1.x == p2.x:  # Vertical line
                x = self.x_map[p1.x]
                if x >= width:
                    continue

                y_start, y_end = sorted([self.y_map[p1.y], self.y_map[p2.y]])
                for y in range(y_start, y_end):
                    cells[y][x] = not cells[y][x]

        # Fill areas based on boundaries and calculate sums
        self.sums: list[list[int]] = [[0] * (width + 1) for _ in range(height + 1)]
        for y in range(height):
            is_inside = False
            # The uncompressed dimensions are needed for the area calculation
            area_height = self.ys[y + 1] - self.ys[y]
            for x in range(width):
                if cells[y][x]:
                    is_inside = not is_inside

                area_width = self.xs[x + 1] - self.xs[x]
                self.sums[y + 1][x + 1] = (
                    self.sums[y + 1][x]
                    + self.sums[y][x + 1]
                    - self.sums[y][x]
                    + (area_width * area_height if is_inside else 0)
                )

    def in_bounds(self, pos: vector2) -> bool:
        return pos.x in self.x_map and pos.y in self.y_map

    def filled_area(self, a: vector2, b: vector2) -> int:
        x_start, x_end = sorted([self.x_map[a.x], self.x_map[b.x]])
        y_start, y_end = sorted([self.y_map[a.y], self.y_map[b.y]])

        area = a.rect_area(b)
        gap_area: int = (
            self.sums[y_end][x_end]
            - self.sums[y_start][x_end]
            - self.sums[y_end][x_start]
            + self.sums[y_start][x_start]
        )
        expected_gap = (self.xs[x_end] - self.xs[x_start]) * (
            self.ys[y_end] - self.ys[y_start]
        )

        return area if gap_area == expected_gap else 0

In [ ]:
def part_2() -> None:
    with open("..\\data\\09.txt") as file:
        data: list[list[str]] = [line.strip().split(",") for line in file.readlines()]

    points: list[vector2] = [vector2(int(x), int(y)) for x, y in data]
    polygon = Compressed_Grid(points)

    max_area: int = 0
    max_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    for i, p1 in enumerate(points):
        for p2 in points[i + 1 :]:
            area = polygon.filled_area(p1, p2)
            if area <= max_area:
                continue

            max_area = area
            max_pair = (p1, p2)

    print(
        f"Part 2: Max filled area is {max_area} between {max_pair[0]} and {max_pair[1]}"
    )


part_2()

Part 2: Max filled area is 1452422268 between (4615, 66437) and (94737, 50322)
